In [1]:
from dotenv import load_dotenv
from langchain.tools import tool
from langchain.messages import HumanMessage
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from typing import Dict, Any
from tavily import TavilyClient
from pprint import pprint

In [2]:
load_dotenv()

True

# Without web search


In [3]:
model = init_chat_model(model="gemini-2.5-flash", model_provider="google_genai")
agent = create_agent(model=model)

In [4]:
question = HumanMessage(content="How up to date is your training knowldage?")

response = agent.invoke({"messages":[question]})
response["messages"][-1].content

"My knowledge cutoff is generally around **early 2023**.\n\nThis means I can provide information and answer questions based on data and events up to that point. I don't have real-time access to the internet or knowledge of events, news, or developments that have occurred since then.\n\nTherefore, I won't be able to provide information on very recent events, new scientific discoveries, current cultural trends, or breaking news from after early 2023."

# Add web search tool

In [5]:
tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""
    return tavily_client.search(query=query)

web_search.invoke("Who is the current mayor of San Francisco?")

{'query': 'Who is the current mayor of San Francisco?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://en.wikipedia.org/wiki/Mayor_of_San_Francisco',
   'title': 'Mayor of San Francisco - Wikipedia',
   'content': 'The current mayor is Democrat Daniel Lurie.',
   'score': 0.9581049,
   'raw_content': None},
  {'url': 'https://en.wikipedia.org/wiki/Daniel_Lurie',
   'title': 'Daniel Lurie - Wikipedia',
   'content': 'Daniel Lawrence Lurie (born 4 February 1977) is an American politician and philanthropist who is the 46th and current mayor of San Francisco, serving since 2025',
   'score': 0.9284155,
   'raw_content': None},
  {'url': 'https://apnews.com/article/san-francisco-new-mayor-liberal-city-81ea0a7b37af6cbb68aea7ef5cc6a4f0',
   'title': "San Francisco's new mayor is starting to unite the fractured city",
   'content': 'San Francisco Mayor Daniel Lurie, a political newcomer and Levi Strauss heir, has marked his first 100 days with a han

In [6]:
agent = create_agent(
    model=model,
    tools=[web_search]
)

question = HumanMessage(content="Who is the current mayor of San Francisco?")

response = agent.invoke({"messages": [question]})

In [7]:
response["messages"]

[HumanMessage(content='Who is the current mayor of San Francisco?', additional_kwargs={}, response_metadata={}, id='f2df45b7-04fa-44cb-be4f-5802c79450f2'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'web_search', 'arguments': '{"query": "current mayor of San Francisco"}'}, '__gemini_function_call_thought_signatures__': {'cba5fc4a-5d2b-4146-acc3-63d642989767': 'CqcCAXLI2nymup983xM7Hw0UJtBu7cKqSJCcDTq06ywehhb9PsCQLov5IcaJwCvYdV4Xv+CSwBrJeQY2A9OnWNXZjZluxp4LDjJkMDN/ueBXoABJJWZKNPWBq24QIKZ9r4u4Jcnmd8RvzlSmoXheMJCJsA/S4QJ0W0/uZfKI5Kst3JKy76y9ghyDvws+o8fmBa31RWhFzFXhElT2jv+/C8V7fJYmPGgD6mYEW5btso2gDrapyjbjHdLTStoatMcHFFCIbyJa/eqyeKOoOymDg/UVx9uR0Aryvq9zXUT+9gwGjwfCss938//sworslt5X/EB1thCqD9z+T+MPkXqIcK0+NBJJYxeZUqzzAGP8L6CD5dR/Qu6zsXX3w8jlUm0AkPbcprpgivpT7A=='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b8c7c-9129-7583-b58a-50b82a8d0417-0', tool_calls=[{'name'

In [13]:
response["messages"][-1]

AIMessage(content=[{'type': 'text', 'text': 'The current mayor of San Francisco is Daniel Lurie.', 'extras': {'signature': 'CtEBAXLI2nynk/XHtiq2Gc50bHZAs+SXOQK57byWiOt2VlwpbvFJ9+V/P3EbIT6LZ9Gs+Gew+Oyfp8sfcCuyqarK+b0cVIr3qTt7BJ86I4KMOjl9md2joj/eVK+j8vuc8dwkJIdtoMqN6iDcFuFMZ5HLzHY6BdY6BYq2OQ55yt41h03Se1VTKuh1GCUDTbHK8HHA4CpeOF4+lbsZJWwItqFcUSc2zfTPQMGvTFVmzwORBazydlDlkO7fzdKDHla3cdkrBIbXbfqGXELyueEoOXMSayI='}}], additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b8c7c-9dfb-7031-bffe-22045e7046f2-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 647, 'output_tokens': 44, 'total_tokens': 691, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 33}})

In [14]:
response["messages"][-1].content[0]["text"]

'The current mayor of San Francisco is Daniel Lurie.'